## Обзор данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

In [2]:
# сделаем так, чтобы jupyter не выдавал ошибки
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('users_behavior.csv')

display(df.info())
display(df.head())
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В таблице 5 столбцов. Согласно документации к данным:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

Посмотрим на уникальные значения столбца `is_ultra`:

In [4]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

Похоже, пользователей тарифа "Смарт" гораздо **больше**.  

Математическое описание всех данных выдало следующее:
* По звонкам - в среднем **63**, медина равна **62**, СО **33**
* По длительности - в среднем **439**, медиана равна **431**, СО **235**
* Количеству СМС - в среднем **38**, медиана равна **30**, СО **36**
* Интернет трафику - в среднем **17208**, медиана равна **16944**, СО **7571**

-



Посмотрим на математическое описание данных, разделенных по тарифам:

In [5]:
df.query('is_ultra == 0').describe()

,calls,minutes,messages,mb_used,is_ultra
count,2229.000000,2229.000000,2229.000000,2229.000000,2229.0
mean,58.463437,405.942952,33.384029,16208.466949,0.0
std,25.939858,184.512604,28.227876,5870.498853,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0
25%,40.000000,274.230000,10.000000,12643.050000,0.0
50%,60.000000,410.560000,28.000000,16506.930000,0.0
75%,76.000000,529.510000,51.000000,20043.060000,0.0
max,198.000000,1390.220000,143.000000,38552.620000,0.0


У пользователей "Смарт":
* По звонкам - в среднем **58**, медина равна **60**, СО **26**
* По длительности - в среднем **406**, медиана равна **411**, СО **185**
* Количеству СМС - в среднем **33**, медиана равна **28**, СО **28**
* Интернет трафику - в среднем **16209**, медиана равна **20044**, СО **5871**

In [6]:
df.query('is_ultra == 1').describe()

,calls,minutes,messages,mb_used,is_ultra
count,985.000000,985.000000,985.000000,985.000000,985.0
mean,73.392893,511.224569,49.363452,19468.823228,1.0
std,43.916853,308.031100,47.804457,10087.178654,0.0
min,0.000000,0.000000,0.000000,0.000000,1.0
25%,41.000000,276.030000,6.000000,11770.280000,1.0
50%,74.000000,502.550000,38.000000,19308.010000,1.0
75%,104.000000,730.050000,79.000000,26837.720000,1.0
max,244.000000,1632.060000,224.000000,49745.730000,1.0


"Ультра":
* По звонкам - в среднем **73**, медина равна **74**, СО **44**
* По длительности - в среднем **512**, медиана равна **503**, СО **309**
* Количеству СМС - в среднем **49**, медиана равна **38**, СО **48**
* Интернет трафику - в среднем **19469**, медиана равна **19308**, СО **10088**

##### Вывод

В данных содержится информация о поведении абонентов компании "Мегалайн": количество сделанных ими звонков, их длительность, количество СМС и потребление интернет-трафика. Временной период не определен. Известно, что им доступно два тарифа - "Смарт" и "Ультра". В ходе обзора данных выяснилось, что пользователей "Смарта" больше.
Математическое описание данных, разбитых по тарифам, показало, что бóльшее потребление связи приходится на пользователей тарифа "Ультра". Также было представлено математическое описание по всем данным. 
Ошибок в данных не обнаружилось. 

## Разделение данных на выборки

В этом пункте разобьём модель на три разные выборки: обучающую, валидационную и тестовую. Соотношение будет равно **3:1:1**, так как спрятанной тестовой выборки у нас нет. Для начала поделим общий датафрейм на обучающую и валидационную выборку:

In [7]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

Теперь разделим валидационную выборку пополам, чтобы определить тестовую:

In [8]:
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

Теперь создадим переменные для признаков и целевого признака:

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

Проверим качество проведенного деления данных:

In [10]:
features_targets = [features_train, target_train, features_valid, target_valid, features_test, target_test]

for var in features_targets:
    display(var.shape)

(1928, 4)

(1928,)

(643, 4)

(643,)

(643, 4)

(643,)

##### Вывод

В данном пункте мы разбили все данные на 3 разные выборки и определили для каждой признаки и целевой признак. 

## Исследование моделей

В данном пункте мы исследуем несколько разных моделей и выберем лучшую, чтобы проверить её на тестовой выборке. Перед нами стоит задача классификации, поэтому будут рассмотрены: "Дерево решений", "Случайный лес" и "Логистическая регрессия". В качестве метрики качества используем `accuracy_score`. Сначала посмотрим на качество модели без смены гиперпараметров. Затем поменяем параметры и сравним результаты. 

### Дерево решений

In [11]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print("Accuracy дерева решений:", accuracy)

Accuracy дерева решений: 0.713841368584759


In [12]:
# смена гипермпараметров в цикле
best_model = None
best_result = 0

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        max_depth = depth
        best_model = model
        best_result = result
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result, ", максимальная глубина =", max_depth)        

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 , максимальная глубина = 3


### Случайный лес

In [13]:
model = RandomForestClassifier(random_state=12345) 
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print("Accuracy случайного леса:", accuracy)

Accuracy случайного леса: 0.7853810264385692


In [14]:
# смена гипермпараметров в цикле
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        max_est = est
        best_model = model 
        best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result, ", количество наблюдателей =", max_est)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692 , количество наблюдателей = 10


### Логистическая регрессия 

In [15]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
print("Accuracy логистической регрессии:", accuracy)

Accuracy логистической регрессии: 0.7853810264385692


In [16]:
# смена гипермпараметров в цикле
best_model = None
best_result = 0
for iters in range(100, 1000, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iters)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        max_iter = iters
        best_model = model 
        best_result = result 

print("Accuracy наилучшей модели на валидационной выборке:", best_result, ", количество итераций =", max_iter)

Accuracy наилучшей модели на валидационной выборке: 0.7107309486780715 , количество итераций = 100


##### Вывод

Мы провели исследование трех разных моделей и выяснили следующее: Дерево решений и Случайный лес выдали одинаковое значение accuracy - **0.7853810264385692**, самое низкое значение показала логистическая регрессия - **0.7107309486780715**. При выборе модели для проведения тестировнаия обратимся к скорости работы обеих. Дерево решений выдало такой же результат при глубине в **3**, как Случайный лес при **10** наблюдателях, значит, скорость работы Дерева решений выше, его и будем использовать на тестовой выборке. 

## Проверка модели на тестовой выборке

In [17]:
model = DecisionTreeClassifier(random_state=12345, max_depth=3)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
print("Accuracy модели на тестовой выборке при глубине в 3:", accuracy)

Accuracy модели на тестовой выборке при глубине в 3: 0.7791601866251944


##### Вывод

Получается, что accuracy Дерева решений при глубине в 3 равна **0.7791601866251944**.

## Проверка модели на адекватность

Нужно проверить имеет ли модель предсказательную силу и справляется ли она с задачей лучше, чем если бы мы предсказывали случайным образом или каким-нибудь константным значением. Нужно сравнить показатели метрик модели с **DummyClassifier**. DummyClassifier - это классификатор, который делает прогнозы, используя простые правила. Этот классификатор полезен в качестве простой основы для сравнения с другими (реальными) классификаторами.

In [18]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_test, target_test)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_test)
dummy_clf.score(features_test, target_test)

0.6842923794712286

##### Вывод

Доля правильных ответов в DummyClassifier равна **0.6842923794712286**, что ниже выбранной модели

## Общий вывод

Был проведен обзор данных об абонентах компании "Мегалайн", использующих два внутренних тарифа -  "Смарт" и "Ультра". Ошибок в данных не было обнаружено. Перед нами встала задача классификации, для решения которой мы рассмотрели 3 разные модели: Дерево решений, Случайный лес и Логистистическую регрессию. Для улучшения качества результатов предсказания каждой модели мы меняли гипермпараметры в цикле: максимальную глубину, количество наблюдателей, количество итераций. Выяснилось, что Дерево решений и Случайный лес показали одинаковую долю правильных ответов. Для определения подходящей модели для исследования на тестовой выборке мы обратили внимание на скорость - Дерево решений показало лучший резльтат при глубине в 3, тогда как Случайный лес выдал то же значение при 10 наблюдателях. Также была произведена проверка на адекватность при помощи DummyClassifier, для этого мы сравнили метрики: на тестовой выборке DummyClassifier показал **0.6842923794712286**, а модель DecisionTreeClassifier **0.7791601866251944**. Порог решения задачи может считаться пройденым.